In [1]:
import requests
import pandas as pd
import numpy as np
from pulp import LpProblem, LpVariable, LpMaximize, lpSum, LpBinary

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

df_players = pd.read_parquet('df_players.parquet')
df_positions = pd.read_parquet('df_positions.parquet')
df_teams = pd.read_parquet('df_teams.parquet')

In [2]:
# Supposons que df_players est votre DataFrame préparé
# Il doit contenir les colonnes : 'position', 'price', 'points_per_cost', 'minutes', 'selected_by_percent'

# Exemple initialisation ou import réel:
# df_players = pd.read_csv('votre_fichier.csv')
# df_players['selected_by_percent'] = df_players['selected_by_percent'].astype(float)

budget = 1500  # Exemple budget total (en 0.1M £)
alpha = 0    # Pondération ownership (à ajuster: plus grand = plus pénalisant)

df_players = df_players.reset_index(drop=True)  # Indices alignés

# Créer problème d'optimisation
prob = LpProblem("FPL_Team_Optimizer", LpMaximize)

# Variables binaires pour chaque joueur
player_vars = [LpVariable(f"player_{i}", cat=LpBinary) for i in range(len(df_players))]

# Fonction objectif : maximiser points_per_cost tout en minimisant selected_by_percent
prob += lpSum([
    df_players.loc[i, 'points_per_cost'] * player_vars[i] - 
    alpha * df_players.loc[i, 'selected_by_percent'] * player_vars[i]
    for i in range(len(df_players))
])

# Contraintes d’effectif
prob += lpSum(player_vars) == 15
prob += lpSum([player_vars[i] for i in range(len(df_players)) if df_players.loc[i, 'position'] == 'GKP']) == 2
prob += lpSum([player_vars[i] for i in range(len(df_players)) if df_players.loc[i, 'position'] == 'DEF']) == 5
prob += lpSum([player_vars[i] for i in range(len(df_players)) if df_players.loc[i, 'position'] == 'MID']) == 5
prob += lpSum([player_vars[i] for i in range(len(df_players)) if df_players.loc[i, 'position'] == 'FWD']) == 3

# Contrainte : max 3 joueurs par équipe
for club in df_players['team_name'].unique():
    prob += lpSum([
        player_vars[i]
        for i in range(len(df_players))
        if df_players.loc[i, 'team_name'] == club
    ]) <= 3

# Contraintes budget
prob += lpSum([df_players.loc[i, 'price'] * player_vars[i] for i in range(len(df_players))]) <= budget

# Contrainte minimum minutes (exemple)
prob += lpSum([df_players.loc[i, 'minutes'] * player_vars[i] for i in range(len(df_players))]) >= 1000

# Résoudre
prob.solve()

# Extraire liste des joueurs sélectionnés
selected_players = [df_players.loc[i] for i in range(len(df_players)) if player_vars[i].varValue == 1]
selected_df = pd.DataFrame(selected_players)

# Afficher résultats clés
print(selected_df[['web_name', 'position', 'price', 'points_per_cost', 'minutes', 'selected_by_percent']])


Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /usr/local/python/3.12.1/lib/python3.12/site-packages/pulp/apis/../solverdir/cbc/linux/i64/cbc /tmp/e87045fb6c4644ec9511ac62f40814c7-pulp.mps -max -timeMode elapsed -branch -printingOptions all -solution /tmp/e87045fb6c4644ec9511ac62f40814c7-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 32 COLUMNS
At line 5321 RHS
At line 5349 BOUNDS
At line 6092 ENDATA
Problem MODEL has 27 rows, 742 columns and 3389 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 10.4358 - 0.00 seconds
Cgl0004I processed model has 27 rows, 622 columns (622 integer (551 of which binary)) and 2909 elements
Cbc0038I Initial state - 0 integers unsatisfied sum - 0
Cbc0038I Solution found of -10.4358
Cbc0038I Cleaned solution of -10.4358
Cbc0038I Before mini branch and bound, 622 integers at bound fixed and 0 continuous
Cbc003

In [3]:
selected_df.sort_values(by = ['position'])

,first_name,web_name,team_short,team_name,position,price,total_points,points_per_cost,points_per_minutes,fdr_next_6,ict_index,selected_by_percent,selected_rank,form,minutes,transfers_in,transfers_in_event,transfers_out,transfers_out_event,assists,goals_scored,goal_involvements,expected_goals,expected_assists,expected_goal_involvements,GI_on_xGI,expected_goals_per_90,saves_per_90,expected_assists_per_90,expected_goal_involvements_per_90,expected_goals_conceded_per_90,goals_conceded_per_90,defensive_contribution_per_90,minutes_last,xGI_last,xG_last,ict_last,total_points_last,total_points_last_per_xGI_last
0,Omar,Alderete,SUN,Sunderland,DEF,40.0,39.0,0.975000,0.080082,3.000000,29.8,3.4,100,8.5,487.0,330585,100506,69730,2336,1.0,1.0,2.0,0.58,0.33,0.91,2.197802,0.11,0.00,0.06,0.17,1.21,0.74,10.90,90.00,0.1775,0.1300,5.325,8.50,47.887324
1,Marcos,Senesi,BOU,Bournemouth,DEF,48.0,44.0,0.916667,0.081481,3.166667,28.1,19.6,19,8.2,540.0,2298568,176357,176763,4007,2.0,0.0,2.0,0.14,0.37,0.51,3.921569,0.02,0.00,0.06,0.08,0.91,1.17,13.50,90.00,0.1125,0.0350,5.250,8.25,73.333333
2,Marc,Guéhi,CRY,Crystal Palace,DEF,48.0,43.0,0.895833,0.079630,3.166667,23.1,27.0,10,8.8,540.0,2080853,180345,414063,4271,2.0,1.0,3.0,0.39,0.09,0.48,6.250000,0.06,0.00,0.02,0.08,1.14,0.50,7.83,90.00,0.0900,0.0700,4.700,8.75,97.222222
4,Tino,Livramento,NEW,Newcastle,DEF,51.0,37.0,0.725490,0.070342,3.000000,14.3,9.7,45,6.0,526.0,915244,32711,268540,14516,1.0,0.0,1.0,0.00,0.33,0.33,3.030303,0.00,0.00,0.06,0.06,0.65,0.51,3.08,86.50,0.0250,0.0000,1.300,6.00,240.000000
5,Joe,Rodon,LEE,Leeds,DEF,40.0,29.0,0.725000,0.053704,2.666667,20.7,4.8,75,5.0,540.0,457228,32395,151229,3674,0.0,1.0,1.0,0.33,0.01,0.34,2.941176,0.06,0.00,0.00,0.06,1.22,1.50,9.33,90.00,0.0850,0.0825,4.075,5.00,58.823529
39,Igor Thiago,Thiago,BRE,Brentford,FWD,60.0,30.0,0.500000,0.060729,3.166667,35.2,2.1,129,5.5,494.0,267444,53166,152674,6123,0.0,4.0,4.0,1.99,0.16,2.15,1.860465,0.36,0.00,0.03,0.39,1.51,1.82,5.10,78.50,0.2500,0.2150,6.125,5.50,22.000000
46,Wilson,Isidor,SUN,Sunderland,FWD,56.0,27.0,0.482143,0.084639,3.000000,25.2,2.8,110,5.2,319.0,369752,29852,88482,9462,0.0,3.0,3.0,1.23,0.03,1.26,2.380952,0.35,0.00,0.01,0.36,1.37,0.85,3.67,69.50,0.3050,0.2975,4.925,5.25,17.213115
48,João Pedro,João Pedro,CHE,Chelsea,FWD,78.0,37.0,0.474359,0.072549,2.666667,40.2,66.9,1,5.0,510.0,2684288,29828,1182368,159924,3.0,2.0,5.0,1.50,0.23,1.73,2.890173,0.26,0.00,0.04,0.30,1.27,1.24,4.94,87.00,0.2275,0.2025,5.850,5.00,21.978022
3,Robin,Roefs,SUN,Sunderland,GKP,45.0,39.0,0.866667,0.072222,3.000000,19.8,3.1,105,7.5,540.0,354490,47176,90332,2368,0.0,0.0,0.0,0.00,0.01,0.01,0.000000,0.00,3.33,0.00,0.00,1.15,0.67,0.00,90.00,0.0025,0.0000,4.100,7.50,3000.000000
8,Nick,Pope,NEW,Newcastle,GKP,50.0,34.0,0.680000,0.062963,3.000000,13.5,7.1,55,5.2,540.0,527340,43864,199573,5843,1.0,0.0,1.0,0.00,0.00,0.00,0.000000,0.00,2.50,0.00,0.00,0.78,0.83,0.00,90.00,0.0000,0.0000,2.450,5.25,0.000000


In [4]:
selected_df['price'].sum()

np.float64(806.0)

In [5]:
selected_df['selected_by_percent'].sum()

np.float64(212.20000000000002)